In [1]:
!pip install langchain

  Using cached langchain-0.0.231-py3-none-any.whl (1.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.6/197.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.5 MB/s eta 0:00:00a 0:00:01m
  Using cached aiohttp-3.8.4-cp39-cp39-macosx_10_9_x86_64.whl (360 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached dataclasses_json-0.5.9-py3-none-any.whl (26 kB)
  Using cached langchainplus_sdk-0.0.20-py3-none-any.whl (25 kB)
  Using cached numexpr-2.8.4-cp39-cp39-macosx_10_9_x86_64.whl (99 kB)
  Using cached numpy-1.25.1-cp39-cp39-macosx_10_9_x86_64.whl (20.1 MB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached pydantic-1.10.11-cp39-cp39-macosx_10_9_x86_64.whl (2.9 MB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import constants 
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
